# Example Script for the Senior Model

This is the example script of the Senior model and is based on the previous exploration of the teacher, tutor, junior  script. Note that in order to run this Notebook, ray needs to be working !

In [1]:
import os
import grid2op
from lightsim2grid import LightSimBackend
import numpy as np
from pathlib import Path
import ray
import ray.tune as tune
import tensorflow as tf
from ray.tune.schedulers import PopulationBasedTraining
from ray.rllib.models import ModelCatalog
import pickle
import random
import json
example_path = os.getcwd()

Again we test the Grid2Op Environment. Additionally, we already init() ray

In [2]:
backend = LightSimBackend()
env_path = test_data_path = Path(example_path)/ "data" / "training_data_track1"
print(env_path)
env = grid2op.make(env_path, backend=backend)
env.set_id(1)
env.reset()
env.chronics_handler.get_name()
obs = env.get_obs()

if ray.is_initialized:
    ray.shutdown()
ray.init()

c:\Users\thoma\MA\curriculumagent\jupyter_notebooks\data\training_data_track1


c:\Users\thoma\anaconda3\envs\MA1\lib\site-packages\lightsim2grid\gridmodel\_aux_add_trafo.py:67: UserWarning: There were some Nan in the pp_net.trafo["tap_step_degree"], they have been replaced by 0
  warnings.warn("There were some Nan in the pp_net.trafo[\"tap_step_degree\"], they have been replaced by 0")
c:\Users\thoma\anaconda3\envs\MA1\lib\site-packages\lightsim2grid\gridmodel\_aux_add_slack.py:114: UserWarning: We found either some slack coefficient to be < 0. or they were all 0.We set them all to 1.0 to avoid such issues
  warnings.warn("We found either some slack coefficient to be < 0. or they were all 0."
c:\Users\thoma\anaconda3\envs\MA1\lib\site-packages\grid2op\Backend\backend.py:1848: RuntimeWarning: divide by zero encountered in divide
  p_s[sh_conn] *= (self._sh_vnkv[sh_conn] / sh_v[sh_conn]) ** 2
c:\Users\thoma\anaconda3\envs\MA1\lib\site-packages\grid2op\Backend\backend.py:1848: RuntimeWarning: invalid value encountered in multiply
  p_s[sh_conn] *= (self._sh_vnkv[sh_

Python version:,3.9.19
Ray version:,2.5.1
Dashboard:,http://127.0.0.1:8265


(raylet) c:\Users\thoma\anaconda3\envs\MA1\lib\site-packages\ray\dashboard\modules\reporter\reporter_agent.py:56: UserWarning: `gpustat` package is not installed. GPU monitoring is not available. To have full functionality of the dashboard please install `pip install ray[default]`.)
(raylet)   warnings.warn(
(PPO pid=28980) 2024-05-06 15:33:58,594	WARNING algorithm_config.py:643 -- Cannot create PPOConfig from given `config_dict`! Property __stdout_file__ not supported.
(PPO pid=28980) 2024-05-06 15:33:58,596	INFO algorithm.py:536 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(RolloutWorker pid=9496) c:\Users\thoma\anaconda3\envs\MA1\lib\site-packages\lightsim2grid\gridmodel\_aux_add_trafo.py:67: UserWarning: There were some Nan in the pp_net.trafo["tap_step_degree"], they have been replaced by 0
(RolloutWorker pid=9496)   warnings.warn("There were some Nan in the pp_net.trafo[\"tap_step_degree\"], they have been repl

## Training based on the more complex Junior model.

In this example, we show the implementation of the hyper-paramter optimized Junior model by using the
```AdvancedCustomModel```. Note that  one can simply use the ```Grid2OpCustomModel``` if the simple Junior was trained.

In [3]:
with open(Path(example_path) / "data" /'scaler_junior.pkl', "rb") as fp:   #Pickling
    scaler = pickle.load(fp)

c:\Users\thoma\anaconda3\envs\MA1\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.2.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Under consideration that ray has sometimes trouble finding the scaler with one of its workers, we advice to use the ```ray.put()``` and ```ray.get()``` option.

In [4]:
ray_scal = ray.put(scaler)

Now we have to initialize both the config of the environment, as well as of the model

In [5]:
action_path1 = Path(example_path) / "data" / "actionspace_nminus1.npy"
action_path2 = Path(example_path) / "data" / "actionspace_tuples.npy"
action_set = [action_path1, action_path2]
model_path = Path(example_path) / "data" / "junior_model"
data_path = Path(example_path) / "data"  / "junior_experience"
target = Path(example_path) / "data" / "junior_model"/ "hyperparam"
hyperparam_path = Path(example_path) / "data" / 'junior_best_params.json'


In [7]:

# Again define best_params
with open(hyperparam_path) as json_file:
    best_params = json.load(json_file)

best_params["epochs"] = 1
best_params["initializer"] = tf.keras.initializers.Orthogonal()
for name in ["layer1","layer2","layer3","layer4"]:
        best_params[name] = np.round(best_params[name])

print(best_params)


{'activation': 'relu', 'batchsize': 256, 'dropout1': 0.19492379978847293, 'dropout2': 0.1259373123876904, 'initializer': <keras.initializers.initializers.Orthogonal object at 0x0000020ACC5B2190>, 'layer1': 1415.0, 'layer2': 1252.0, 'layer3': 1485.0, 'layer4': 232.0, 'learning_rate': 0.0003013467240068926, 'TRIAL_BUDGET': 11, 'epochs': 1}


In [14]:

# Env Config
env_config = {"action_space_path":action_set,
              "env_path": env_path,
              "action_threshold":0.95,
              'filtered_obs':True,
              'scaler': ray_scal,
              'subset' : False
              }

# Model Config:
model_config = {"path_to_junior":target,
                "custom_config":best_params}

Testing whether we can load the model. Then, we initialize the model config

In [9]:
from curriculumagent.senior.rllib_execution.senior_env_rllib import SeniorEnvRllib
from curriculumagent.senior.rllib_execution.senior_model_rllib import Grid2OpCustomModel

## Testing both the environment and the model

Scip these methods if you want to start directly with the ray optimization.

Note that you can check, whether your scaler works.

In [15]:
# Test Environment:
env =  SeniorEnvRllib(env_config)
print(env.action_space.n)
print(env.observation_space.shape)


c:\Users\thoma\anaconda3\envs\MA1\lib\site-packages\lightsim2grid\gridmodel\_aux_add_trafo.py:67: UserWarning: There were some Nan in the pp_net.trafo["tap_step_degree"], they have been replaced by 0
  warnings.warn("There were some Nan in the pp_net.trafo[\"tap_step_degree\"], they have been replaced by 0")
c:\Users\thoma\anaconda3\envs\MA1\lib\site-packages\lightsim2grid\gridmodel\_aux_add_slack.py:114: UserWarning: We found either some slack coefficient to be < 0. or they were all 0.We set them all to 1.0 to avoid such issues
  warnings.warn("We found either some slack coefficient to be < 0. or they were all 0."
c:\Users\thoma\anaconda3\envs\MA1\lib\site-packages\grid2op\Backend\backend.py:1848: RuntimeWarning: divide by zero encountered in divide
  p_s[sh_conn] *= (self._sh_vnkv[sh_conn] / sh_v[sh_conn]) ** 2
c:\Users\thoma\anaconda3\envs\MA1\lib\site-packages\grid2op\Backend\backend.py:1848: RuntimeWarning: invalid value encountered in multiply
  p_s[sh_conn] *= (self._sh_vnkv[sh_

806
(1429,)


In [16]:

# Run "simulation":
done = False
while done==False:
    act = random.choice(np.arange(env.action_space.n))
    obs,b,done,d = env.step(act)
    print(act,done,b,env.step_in_env,max(obs),min(obs),env.single_env.nb_time_step)

ValueError: too many values to unpack (expected 4)

model:

In [17]:
# Test Model:
model = Grid2OpCustomModel(obs_space = env.observation_space,
                           action_space = env.action_space,
                           num_outputs = env.action_space.n,
                           model_config = {},
                           path_to_junior =target,
                           custom_config = best_params,
                           name="Junior")
print(model.base_model.summary())
obs_dict = {"obs": obs.reshape(1,-1)}
out = model.forward(input_dict = obs_dict, state=1, seq_lens=None)
out

KeyError: <keras.initializers.initializers.Orthogonal object at 0x0000020ACC5B2190>

## Run Rllib experiment to collect the Senior Model

We run the senior experiment. Please adjust the number of cpus, gpus and workers based on your computational power.

In [19]:
# Register the model
ModelCatalog.register_custom_model('Senior',  Grid2OpCustomModel)

In [20]:
NUM_WORKER = os.cpu_count() - 1

In [21]:
pbt = PopulationBasedTraining(
        time_attr="training_iteration",
        metric="episode_reward_mean",
        mode="max",
        perturbation_interval=50,
        resample_probability=0.5,
        # Specifies the mutations of these hyperparams
        hyperparam_mutations={
            "lr": [1e-3, 5e-4, 1e-4, 5e-5, 1e-5],
            "num_sgd_iter": lambda: random.randint(3, 10),
            "vf_loss_coeff": lambda: random.uniform(0.5,1),
            "clip_param": lambda: random.uniform(0.01, 0.5),
            "gamma": lambda: random.uniform(0.975, 1),
            "entropy_coeff": lambda: 10**-random.uniform(2,5)
        })

tune.run(
    "PPO",
    checkpoint_freq=5,
    scheduler= pbt,
    keep_checkpoints_num =30, 
    verbose = 1,
    max_failures=3, 
    num_samples=1,
    local_dir="~/ray_results",
    stop={"training_iteration": 1},
    config={
        "env":  SeniorEnvRllib,
        "env_config":env_config,
        "num_workers":NUM_WORKER,
        "num_envs_per_worker": 1,
        "lr": 5e-5,
        "num_gpus":0.0,
        "num_cpus_per_worker": 1,
        "remote_worker_envs": False,
        "model":{"custom_model":  "binbinchen","custom_model_config": model_config},
    },
)

2024-05-06 15:35:03,707	ERROR tune_controller.py:873 -- Trial task failed for trial PPO_SeniorEnvRllib_409f2_00000
Traceback (most recent call last):
  File "c:\Users\thoma\anaconda3\envs\MA1\lib\site-packages\ray\air\execution\_internal\event_manager.py", line 110, in resolve_future
    result = ray.get(future)
  File "c:\Users\thoma\anaconda3\envs\MA1\lib\site-packages\ray\_private\auto_init_hook.py", line 18, in auto_init_wrapper
    return fn(*args, **kwargs)
  File "c:\Users\thoma\anaconda3\envs\MA1\lib\site-packages\ray\_private\client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
  File "c:\Users\thoma\anaconda3\envs\MA1\lib\site-packages\ray\_private\worker.py", line 2542, in get
    raise value
ray.exceptions.RayActorError: The actor died because of an error raised in its creation task, ray::PPO.__init__() (pid=28980, ip=127.0.0.1, actor_id=b3aa8c21880e0ac687c1036101000000, repr=PPO)
  File "c:\Users\thoma\anaconda3\envs\MA1\lib\site-packages\ray\rllib\e

In [ ]:
isinstance(ray_scal,ray._raylet.ObjectRef)

## Select and convert checkpoint

After the training is completed, one to collect the saved model

Note, you have to specify the correct checkpoint dir in ```"~/ray_results"```

In [ ]:
from curriculumagent.senior.rllib_execution.convert_rllib_ckpt import load_config,load_and_save_model

In [ ]:
example_dir = "/ray_results/PPO/PPO_SeniorEnvRllib_6c729_00001_1_2022-04-06_15-28-19"
ckpt_path = Path(example_dir) / "checkpoint_000675"/"checkpoint-675"
save_path =  Path(example_path) /"senior_model"

First we load the config of the ray experiment and overwrite the class values with the actual classes:

In [ ]:
config = load_config(ckpt_path)
config['env_config'] = env_config
config["model"]['custom_model_config']['custom_config'] = model_config

Now run and save the model

In [ ]:
load_and_save_model(ckpt_path = ckpt_path,config =config, save_path = save_path, ckpt_nr = 675)

## Submission:

In order to correctly submit a working agent, the tensorflow model now only needs to be placed in a submission
folder, together with the action set. An example can be found in ```my_agent_advanced.py```